# <center>پیدا کردن ربات</center>

<div dir="rtl" style="font-size:18px">
احتمالا کارتون WALL-E را بخاطر دارید. رباتی که توانست زمین را نجات دهد و باقیمانده نسل انسان‌ها را به زمین برگرداند. پس از بازگشت انسان‌ها به زمین و پس از محبوبیت زیاد WALL-E، او دوستان زیادی پیدا کرد. بهترین دوست او WALL-F یکی از بازیگوش‌ترین ربات‌هایی است که تاحالا دیده‌اید.
<br>
یکی از دفعاتی که WALL-F به بیرون رفته بود، راهی به درون یک هزار‌تو میابد. تصویر زیر از او در این هزارتو گرفته شده است.
</div>
<br>
<br>
<center>

![WALL-F Ai-Generated](Assets/robot.jpg)

</center>

<div dir="rtl">
در ادامه میخواهیم با روش‌های مختلفی پیدا کنیم که در حال حاضر ربات در چه خانه‌ای قرار دارد. در هرکدام از تست‌کیس‌هایی که لازم است بررسی کنید. خانه 0 و 0 خانه بالا و چپ هزارتو خواهد بود. در ادامه برای هر خانه مشخص خواهد شد که دیواری به هرکدام از جهات دارد یا خیر. 
به طور مثال عبارت
''R L''
به این معنی است که این خانه فقط به چپ و راست خود راه دارد. همچنین شماره‌گذاری خانه‌ها به صورت زیر است.
در هر مرحله او به شما میگوید که در کدام خانه از جدول است. عددی که ربات به شما میگوید یک عدد تصادفی است که در یک ماتریس به شما داده میشود. عدد سطر 
$i$ 
و ستون 
$j$ 
احتمال این را نشان می‌دهد که ربات در خانه 
$i$ 
باشد اما خانه 
$j$ 
را شما گزارش دهد. این ربات در پایان هر مرحله با احتمال برابر به یکی از خانه‌های مجاور میرود. همینطور خانه شروع این ربات هم توزیعی یکنواخت دارد.
</div>
<br>
<br>
<center>

![grid](Assets/grid.jpg)

</center>

<div dir="rtl">
در ابتدا برای پیاده‌سازی ۲ الگوریتمی که در ادامه خواهید دید به آبجکتی از نوع Grid نیاز خواهید داشت. عملکرد این آبجکت را میتوانید از مسیر Assets/helpers.py ببینید. به طور مثال هزارتو زیر را در نظر بگیرید.
</div>

<center>

![maze](Assets/grid2.jpg)

</center>

<div dir="rtl">
grid متناظر با این هزارتو را میتوانید از طریق تابع get_sample_grid بگیرید و با توابع آن آشنا شوید. در صورت لزوم میتوانید هر تابع دلخواهی به این کلاس اضافه کنید. برای اطلاع از تعداد سطرهای grid میتوانید از grid.n و برای اطلاع از تعداد ستون‌ها میتوانید از grid.m استفاده کنید.
</div>

In [1]:
from Assets.helpers import get_sample_grid


In [2]:
grid = get_sample_grid()
print(grid.moves)

[[['R'], ['L', 'R', 'D'], ['L']], [['R'], ['L', 'R', 'U'], ['L']]]


# Particle Filtering

<div dir="rtl">
در این قسمت باید آخرین خانه‌ای که این ربات در آن قرار دارد را با استفاده از الگوریتم Particle Filtering پیدا کنید. برای اینکار در ابتدا تعداد Particle هایی با تعدادی از اردر تعداد خانه‌های هزارتو بسازید. در پایان هر مرحله جمعیت را با توجه به وزن آنها نصف کنید و ذراتی جدید متناسب با تعداد ذرات درون هرخانه بسازید. در پایان خانه‌ای که بیشترین ذرات در آن قرار دارد را برگردانید.
</div>

In [7]:
import random
import numpy as np
def PF(grid, prob, obs):
    """
    input:

    grid: object of type Grid
    obs: array of ints, each one is an observation

    output: 
    number of the last cell
    """
    s = grid.n * grid.m
    particle = np.random.randint(0, s, size = s * 10)
    for report in obs:
        for p in range(len(particle)):
            neighbours = grid.get_neighbors(particle[p])
            particle[p] = random.choice(neighbours)

        probs = np.array(prob[:, report])
        
        probs = probs[particle]
        probs /= probs.sum()
        
        particle = np.random.choice(particle, size=len(particle), replace=True, p=probs)

    counts = np.bincount(particle)
    return np.argmax(counts)
    

In [8]:
from Assets.helpers import PF_checker
PF_checker(PF)

PF PASSED!


# HMM

<div dir="rtl">
در این قسمت غیر از اینکه باید بفهمیم WALL-F در کدام خانه قرار دارد. میخواهیم بدانیم که محتمل‌ترین مسیری که طی کرده‌ است چه مسیری است. برای اینکار الگوریتم viterbi را پیاده‌سازی کنید.
</div>

In [183]:
def viterbi(grid, prob, obs):
    """
    Viterbi algorithm for finding the most likely sequence of hidden states.

    Parameters:
    - grid: object of type Grid
    - prob: array representing probabilities
    - obs: array of ints, each one is an observation

    Returns:
    - array of cell numbers, same shape as obs
    """
    s = grid.m * grid.n


    transition_mat = np.zeros((s, s))
    for i in range(s):
        for j in grid.get_neighbors(i):
            transition_mat[i, j] = 1
        transition_mat[i] = transition_mat[i] / np.sum(transition_mat[i])


    
    dp = np.zeros((s, len(obs)))
    parent = np.zeros((s, len(obs)), dtype=int)

    dp[:, 0] = prob[:, obs[0]]

    for idx, report in enumerate(obs):
        if idx == 0: continue
        for i in range(s):
            neighbors = grid.get_neighbors(i)
            max_prob = float('-inf')
            max_parent = -1

            for n in neighbors:
                current_prob = dp[n, idx - 1] * transition_mat[n, i]
                if current_prob > max_prob:
                    max_prob = current_prob
                    max_parent = n

            dp[i, idx] = max_prob *  prob[i, report]
            parent[i, idx] = max_parent

    res = [np.argmax(dp[:, -1])]
    idx = len(obs) - 1

    while idx > 0:
        res.append(parent[res[-1], idx])
        idx -= 1
    return res[::-1]


In [184]:
from Assets.helpers import viterbi_checker
viterbi_checker(viterbi)

VITERBI PASSED!
